In [60]:
from pandas_datareader import data as web
import pandas as pd
import datetime as dt
import fix_yahoo_finance as yf
from os import listdir
from os.path import join
class Price_getter:
    def __init__(self,path='data',region='.TW'):  
        yf.pdr_override()
        self.store_path=path
        self.region=region
        self.csv=".csv"
    def get_price(self,code):
        if self._check_csv_exist(code):
            print("Exist .csv!")
            self._check_update(code)
        else:
            print(".csv doesn't exist!")
            print("Download new data!")
            self._get_new_code(code)
        return self.df
        
    def _read_from_csv(self,path):
        self.df=pd.read_csv(path,index_col=0,infer_datetime_format=True)
        self.df.index = pd.to_datetime(self.df.index)
        self.df.dropna(how='any',inplace=True)
        self.df.sort_index()
    def _store_to_csv(self,path):
        self.df.to_csv(path)
    def _check_csv_exist(self,code):
        full_name=code+self.region+self.csv
        files = listdir(self.store_path)
        if full_name in files:
            return True
        return False
    def _check_update(self,code):
        path=join(self.store_path,code+self.region+self.csv)
        self._read_from_csv(path)
        today=dt.datetime.now().date()
        if today>self.df.tail(1).index[0].date():
            start_day=self.df.tail(1).index[0].date()+dt.timedelta(days=1)
            end_day=today
            new_df=self._download_new_data(code,start_day,end_day)
            if self.df.tail(1).index[0].date()!=new_df.head(1).index[0].date():
                print("Obsolete")
                print("Update & save to .csv")
                frames = [self.df, new_df]
                self.df= pd.concat(frames)
                self._store_to_csv(path)
    def _download_new_data(self,code,start_day,end_day):
        df = web.get_data_yahoo([code+self.region],start=start_day,end=end_day)
        return df
    def _get_new_code(self,code):
        import twstock
        start_day = dt.datetime.strptime(twstock.codes[code].start, '%Y/%m/%d')
        self.df=price_getter._download_new_data(code,start_day,dt.datetime.now().date())
        path=join(self.store_path,code+self.region+self.csv)
        self._store_to_csv(path)
if __name__=='__main__':
    price_getter=Price_getter()
    code='6214'
    df=price_getter.get_price(code)
    
        
       
    '''
df.Close.plot()
plt.show()'''

.csv doesn't exist!
Download new data!
[*********************100%***********************]  1 of 1 downloaded
